### Import package

In [42]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os
import math
import pandas as pd
from collections import Counter
from threading import Thread
import time

### Step 1: Merged all downloaded sdf files from pharmit which match pharamcophore:

In [43]:
def mergesdf(outfilename,folder=".\\Hitreport\\",output=".\\"):
    """
    merge all sdf files which download from pharmit
    outfilename was the name for the merge sdf
    the folder were the sdf file stored matched compounds of each frame
    the output was the folder to export the merge sdf
    return None
    """
    files = os.listdir(folder)
    for file in files:  #remove format of file not "sdf" within folder
        if file[-3:] != "sdf":
            files.remove(file)
    
    collectstring = ""  #read all file's contents and save in variable "collectstring"
    for file in files:
        filename = folder+file
        with open (filename,"r") as rf:
            collectstring += rf.read()+"\n"
            rf.close()
    
    with open(output+outfilename,"w") as wf: #output the collectstring data
        wf.write(collectstring)
        wf.close()
    print("merged work is finished!")
    return None

def getcname(filename=".\\sample.sdf"):  
    """
    obtain all compound name which record in merge sdf
    return cnames, a list stored the compounds name String.
    """
    with open(filename,"r") as rf:
        content = rf.readlines()
        rf.close()
    cnames = []
    for line in content:
        line = line.strip("\n")
        if line[-3:] == "sdf":
            cnames.append(line.replace(".sdf",""))
    cnames = list(set(cnames))
    return cnames

### Run the merge work, Make sure the path is normal:

In [44]:
folder = ".\\output\\PPYhit\\"
output = ".\\output\\"
outfilename = output+"merge-PPY.sdf"  

print("The folder stored the hit sdf gotten form Pharmit: ["+folder+"]")
print("The output path is ["+output+"]")
print("The exporting merged sdf file name will be ["+outfilename+"]")

mergesdf(outfilename,folder=folder)
cnames = getcname(filename=outfilename)
with open(output+"cnames.txt","w") as f: 
    for i in cnames:
        f.write("\""+i+"\""+',') 
print(len(cnames),cnames[0:3])  #make sure the list content is non-empty

The folder stored the hit sdf gotten form Pharmit: [.\output\PPYhit\]
The output path is [.\output\]
The exporting merged sdf file name will be [.\output\merge-PPY.sdf]
merged work is finished!
1 ['PPY']


### Step 2: Extract specific molecule's comformers form merged file:

In [45]:
def find_specific_comc(cname,infilename,outfilename):
    """
    Here, the specific compound conformers were extracted from the merge sdf
    based on the cname got from last step.
    return None
    """
    with open(infilename,"r") as f:
        content = f.readlines()
        f.close()
    datas = []
    linenumber = 0
    for line in content:
        if line == cname+".sdf\n":
            start = linenumber
            data = "$$$$\n"
            for line2 in content[start:]:
                data += line2
                if line2 == "M  END\n":
                    break
            datas.append(data) 
        linenumber +=1
    with open(outfilename,"w") as f:
        datas[0] = datas[0].replace("$$$$\n","")
        for data in datas:
            f.write(data)
        f.close()
    return None

### Extract the sdf files (need some time) Make sure your path is correct:

In [46]:
st = time.time()
outfilenames = []
for cname in cnames:
    try:
        extractCname = output +cname+".sdf"
        outfilenames.append(extractCname) 
        find_specific_comc(cname,outfilename,extractCname)
    except Exception as e:
        print(e)
        pass
print("Number of extracted compounds were "+str(len(outfilenames)))
print("Extracted compounds were",outfilenames[0:3])
et = time.time()

print("Time consuming is "+str(round(et-st,4)))

Number of extracted compounds were 1
Extracted compounds were ['.\\output\\PPY.sdf']
Time consuming is 0.008


### Step 3: Align the Melecule's comformer

In [47]:
def alignc(sdfsfilename):
    """
    when the same conformer rotate or move, it maybe hit the pharmacophore
    It possibly cause the conformers of a molecule downloaded from Pharmit appeared duplication.
    Alignment is the first step to screen the different conformer.
    """
    outfilename = sdfsfilename.replace(".sdf","_aligned.sdf")
    mols = [x for x in Chem.SDMolSupplier(sdfsfilename,removeHs=True) if x is not None]
    mcounts = len(mols)
    for i in range(mcounts):
        Chem.rdMolAlign.AlignMol(mols[i],mols[0])
    w = Chem.SDWriter(outfilename)
    for mol in mols:
        w.write(mol)
    w.close()
    return None

In [48]:
st = time.time()
for outfile in outfilenames: #generated from the last step
    alignc(outfile)
alignStoredFiles = os.listdir(output)
alignfilenumber = 0
noalignfilenumber = 0
for file in alignStoredFiles:
    try:
        if file[-3:] == "sdf":
            if file[-12:] == "_aligned.sdf":
                alignfilenumber += 1
            else:
                noalignfilenumber += 1
    except Exception as e:
        print(e)
        pass

print("after aligning compound's number: " + str(alignfilenumber))
print("The export folder is: ",output)
et = time.time()
print("Time consuming is "+str(round(et-st,4)))

after aligning compound's number: 1
The export folder is:  .\output\
Time consuming is 0.024


### Step 4: Merged the sdf file after aligned:

In [50]:
def merge_aligned_sdf(outfilename,folder = ".\\",output=".\\"):
    """
    folder was the path to save the aligned sdf
    the output was the path to export the merged sdf
    the outfilename is the name of the merged sdf
    return None
    """
    files = os.listdir(folder)
    collectfiles = []
    for file in files:  #remove non-sdf files
        try:
            keyword = file.split("_")[-1]
            if keyword[0:7] == "aligned":
                collectfiles.append(file)
        except:
            pass
    collectstring = ""
    for file in collectfiles:
        filename = folder+file
        with open (filename,"r",encoding = "utf8") as rf:
            collectstring += rf.read()+"\n"
            rf.close()
    
    with open(output+outfilename,"w") as wf:
        wf.write(collectstring)
        wf.close()
    print("merged work is finished!")
    return None

In [20]:
m_aglined_fname = "alignedsdfs.sdf" 
# The name of merged files

In [51]:
afp = ".\\"
mfp = ".\\output\\"
print("The aligned sdf were stored in "+afp)
print("The merged sdf will be stored in "+mfp+m_aglined_fname)
merge_aligned_sdf(m_aglined_fname,folder=afp,output=mfp) #execute merged task

The aligned sdf were stored in .\
The merged sdf will be stored in .\output\alignedsdfs.sdf
merged work is finished!



### Step 5: Calculate the RMSD relative to 0 for each comformer in  Merged SDF:

In [52]:
def calculatermsd(compoundname,compoundxyz):  
    """
    Installed into the function collectrmsd0(filename)
    according to the sdf data to calculate the rmsd value relative to the zero position (0,0,0)
    return compoundrmsd, a string include the rmsd value
    """
    N = len(compoundxyz)  #all atoms number
    atomsd = 0 #record the offsets for all atoms
    for xyz in compoundxyz:
        coordinatelist = []
        xyz = list(filter(None,xyz.split(",")))
        for coordinate in xyz:

            coordinatelist.append(float(coordinate.strip(" "))) #a xyz coordination list of a atom which contained 3 values after treated
        atomd = math.pow(coordinatelist[0]-0,2)\
        +math.pow(coordinatelist[1]-0,2)\
        +math.pow(coordinatelist[2]-0,2)/3 #the offset of one atom relative to the (0,0,0)
        atomsd += atomd
        
    compoundrmsd = round(math.sqrt(atomsd/N),3) #calculate the rmsd for a compound relation to the (0,0,0)
    compoundrmsd = compoundname+","+str(compoundrmsd)+"\n"
    return compoundrmsd


def collectrmsd0(filename):
    """
    The rmsd between the conformer and (0,0,0) were calculated
    We just need to get the difference was existed between each conformer
    If both conformers were same,the rmsd value of its is same.
    return compoundsrmsd, a list stored all rmsd value of each conformer for this inputed compound
    """
    with open(filename,"r") as rf:
        content = rf.readlines()
        rf.close()
    compoundname = ""
    compoundxyz = []
    compoundsrmsd = ""
    linenumbers = len(content)
    for index in range(linenumbers):
        line = content[index].strip("\n")
        if line[-3:] == "sdf":
            compoundname += line
        if line[35:48] == "0  0  0  0  0":
            compoundxyz.append(line[2:30].replace("  ",",")) #add line including each atom coordinate 
        if line == "M  END":
            compoundsrmsd += calculatermsd(compoundname,compoundxyz) #obtain this comformer's rmsd and append to the string
            compoundname = ""
            compoundxyz = []
    return compoundsrmsd


def de_weight(compoundsrmsd):
    """
    The same rmsd value were remove
    return compoundsrmsd_, a String
    """
    compoundsrmsd = compoundsrmsd.split("\n")
    compoundsname = []
    for unit in compoundsrmsd:   #collect compound names
        name = unit.split(",")[0]
        compoundsname.append(name)
    compoundsname = set(compoundsname)  #de-weight for compound names
    compoundsrmsd_ = ""
    for name in compoundsname: 
        rmsdvalueslist = []
        for unit in compoundsrmsd:
            unit = unit.split(",")
            if len(unit) != 2:
                continue
            if unit[0] == name:
                rmsdvalueslist.append(unit[1])

        rmsdvalueslist = set(rmsdvalueslist) #de-weight for rmsd
        for rmsd in rmsdvalueslist:
            compoundsrmsd_ += name+","+rmsd+"\n"
    return compoundsrmsd_


def getcfreq(crmsd):
    """
    summary the conformers number
    """
    crmsdl = crmsd.split("\n")
    cnamefreq = []
    for line in crmsdl:
        line = line.split(",")
        cnamefreq.append(line[0].replace(".sdf",""))
    cfrequence = Counter(cnamefreq)
    del crmsdl
    return cfrequence


def writereport(result,filename):
    with open(filename,"w") as wf:
        wf.write("cname,rmsd0,CHA\n")
        for data in result:
            wf.write(data)
        wf.close()
    return None
print("everything is normal!")

everything is normal!


In [53]:
infilename = ".\\output\\" + m_aglined_fname
output = ".\\"
reffilename = ".\\PPY\\" + "pep cfreq report.csv"  
#this file which produced from code named "Calculater of CHA.ipynb"
#It can found the compound not detected in download sdf files

print("The input file is: "+infilename)
print("The output folder will be: "+output)
print("The message of conformers number for each compound were stored in "+reffilename)


The input file is: .\output\alignedsdfs.sdf
The output folder will be: .\
The message of conformers number for each compound were stored in .\PPY\pep cfreq report.csv


In [54]:
datacombine = ""
datacombine += collectrmsd0(infilename)  #calculate rmsd
crmsd = de_weight(datacombine)   #de-weight

print("okay!")

okay!


In [55]:
df = pd.read_csv(reffilename)
df.head()

,cname,frequence
0,PPY,99


In [56]:
fredata = list(df["frequence"])
cnamedata = list(df["cname"])

print("Total number of compounds submitted to the pharmit: ",len(fredata))
outputdata = []
cfrequence = getcfreq(crmsd)
print("Total number of the compound which matched pharmacophore: ",len(cfrequence.keys()))

Total number of compounds submitted to the pharmit:  1
Total number of the compound which matched pharmacophore:  2


### Step 6: Export the CCA report:

In [57]:
reportname = ".\\PPY CCA report.csv"
print("The exporting file name will be: "+reportname)

outputdata = []
for i in range(len(cnamedata)):
    try:
        cname = cnamedata[i]
        statepoint = 0
        for c in cfrequence.keys():
            if c == cname:
                cha = round(cfrequence[c]/fredata[i],3)
                result = cname+","+ str(cfrequence[c])+","+str(cha)+"\n"
                outputdata.append(result)
                statepoint = 1
                break
        if statepoint != 1:
            result = cname+",0,0\n"
            outputdata.append(result)

        if statepoint == 1:
            statepoint = 0
    except Exception as e:
        print(e)

writereport(outputdata,reportname) #export the CCA report

The exporting file name will be: .\PPY CCA report.csv


### Step 7: Remove large number of sdf produced in middle step:

In [37]:
print(outfilenames[0:4])
for oldfile in outfilenames:  #remove the extract sdf file
    try:
        os.remove(oldfile)
        print("file ",oldfile," was removed.")
    except Exception as e:
        print(e)

In [41]:
#removed the sdf file after aligned
folder = ".\\output\\"
files = os.listdir(".\\output\\")
delfiles = []
for file in files:
    try:
        keyword = file.split("_")[-1]
        if file[0:7] == "aligned":
            delfiles.append(file)
    except:
        pass
print("The following files will be remove: ",delfiles)  #Make sure not exist any important file
for file in delfiles:
    file = ".\\" + file
    os.remove(folder+file)

The following files will be remove:  ['alignedsdfs.sdf']
